# Project: Pawfect Planner – Your Personalized Pet Service and Activity Recommender


In [1]:
import requests
import requests_cache
import pandas as pd
from datetime import datetime, timedelta, timezone
from bs4 import BeautifulSoup
import os

In [2]:
# Initialize Caching
# Initialize cache to store API responses for 1 hour
requests_cache.install_cache('pawfect_cache', expire_after=3600)

In [ ]:
# ✅ Set API Keys (Load from environment variables)
# For github, I blur the API information
os.environ["GOOGLE_PLACES_API_KEY"] = 'Google Place API'
os.environ["OPENWEATHERMAP_API_KEY"] = 'Open Weather Map API'
os.environ["YELP_API_KEY"] = "Yelp API"

GOOGLE_PLACES_API_KEY = os.getenv("GOOGLE_PLACES_API_KEY")
OPENWEATHERMAP_API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")
YELP_API_KEY = os.getenv("YELP_API_KEY")

In [4]:
# 🗺️ **Convert City Name or ZIP Code to Latitude & Longitude**
def get_lat_lon_from_location(location):
    geo_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {'address': location, 'key': GOOGLE_PLACES_API_KEY}
    response = requests.get(geo_url, params=params)

    if response.status_code == 200:
        data = response.json()
        if data.get('results'):
            lat_lng = data['results'][0]['geometry']['location']
            return lat_lng['lat'], lat_lng['lng']
    return None, None

In [5]:
def get_weather(location):
    url = "http://api.openweathermap.org/data/2.5/weather"
    if location.isdigit():
        params = {'zip': f"{location},US", 'appid': OPENWEATHERMAP_API_KEY, 'units': 'metric'}
    else:
        params = {'q': location, 'appid': OPENWEATHERMAP_API_KEY, 'units': 'metric'}
        
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        return {
            'City': data.get('name', 'Unknown'),
            'Temperature': data['main']['temp'],
            'Weather': data['weather'][0]['description'].capitalize(),
            'Wind Speed': data['wind']['speed'],
            'Recommendation': get_weather_recommendation(data['main']['temp'], data['weather'][0]['description'])
        }
    return {}
    

In [6]:
# 💡 **Weather-Based Recommendation**
def get_weather_recommendation(temp, condition):
    if 'clear' in condition.lower() and temp > 20:
        return "☀️ Warm & Sunny: Perfect for outdoor dog parks and pet-friendly cafes!"
    elif 'cloud' in condition.lower() and 10 <= temp <= 20:
        return "🌥️ Cloudy & Mild: Great for both indoor and outdoor activities."
    elif 'rain' in condition.lower() or temp < 10:
        return "🌧️ Rainy & Cold: Recommend indoor pet stores or grooming sessions."
    elif 'storm' in condition.lower() or 'snow' in condition.lower():
        return "⚡ Stormy/Snowy: Stay cozy indoors or check virtual pet events."
    return "🌎 General weather: Suitable for most activities."

In [7]:
# 🏡 **Fetch Pet-Friendly Places (Google & Yelp Combined)**
def get_pet_friendly_places(location, radius=5000):
    lat, lon = get_lat_lon_from_location(location)
    if not lat or not lon:
        return pd.DataFrame()

    categories = {
        'Dog Park': 'dog park',
        'Pet Store': 'pet store',
        'Grooming': 'pet grooming',
        'Veterinary Clinic': 'veterinary clinic',
        'Pet-Friendly Café': 'pet-friendly café',
        'Pet-Friendly Hotel': 'pet-friendly hotel'
    }

    places = []

    # ✅ Fetch from **Google Places API**
    for category, keyword in categories.items():
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
            'location': f"{lat},{lon}",
            'radius': radius,
            'keyword': keyword,
            'key': GOOGLE_PLACES_API_KEY
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            for place in data.get('results', []):
                places.append({
                    'Name': place['name'],
                    'Address': place.get('vicinity', 'N/A'),
                    'Rating': place.get('rating', 0),
                    'Category': category,
                    'Source': 'Google'
                })

    # ✅ Fetch from **Yelp API**
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": f"Bearer {YELP_API_KEY}"}

    for category, yelp_category in categories.items():
        params = {
            "location": location,
            "radius": radius,
            "limit": 10,
            "categories": yelp_category
        }
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            for place in data.get('businesses', []):
                places.append({
                    'Name': place['name'],
                    'Address': place.get('location', {}).get('address1', 'N/A'),
                    'Rating': place.get('rating', 'N/A'),
                    'Category': category,
                    'Source': 'Yelp'
                })

    # Convert to DataFrame & limit results to top 5 per category
    places_df = pd.DataFrame(places)
    if not places_df.empty:
        places_df = (
            places_df.sort_values(by=['Category', 'Rating'], ascending=[True, False])
                      .groupby('Category')
                      .head(5)
        )

    return places_df

In [8]:
def get_pet_events(location):
    """
    Fetch the 5 most recent pet-related events from Yelp API, including past events.
    """
    url = "https://api.yelp.com/v3/events"
    headers = {"Authorization": f"Bearer {YELP_API_KEY}"}
    params = {
        "location": location,
        "categories": "pets",
        "limit": 10,  # Fetch more events first, then filter the latest 5
        "sort_on": "time_start",  # ✅ Sort by event start time
        "sort_by": "desc"  # ✅ Descending order (newest events first)
    }

    response = requests.get(url, headers=headers, params=params)
    events = []

    if response.status_code == 200:
        data = response.json()
        for event in data.get('events', []):
            event_date = event.get('time_start', 'Unknown')
            if event_date != 'Unknown':
                events.append({
                    'Name': event['name'],
                    'Date': event_date,
                    'Location': event.get('location', {}).get('city', 'Unknown'),
                    'URL': event.get('event_site_url', 'N/A')
                })

    # ✅ Return only the **5 newest events** (including past events)
    return pd.DataFrame(events[:5]) if events else pd.DataFrame()

In [9]:
import requests

def get_wiki_breed_summary(breed_name):

    page_title = breed_name.replace(" ", "_")
    
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{page_title}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        title = data.get('title', '')
        description = data.get('description', '')
        extract = data.get('extract', '')
        thumbnail = data.get('thumbnail', {}).get('source', '')
        original_image = data.get('originalimage', {}).get('source', '')
        
        return {
            'title': title,
            'description': description,
            'extract': extract,
            'thumbnail': thumbnail,
            'original_image': original_image
        }
    else:
        print(f"Error fetching Wikipedia summary for {breed_name}: {response.status_code}")
        return {}

    
if __name__ == "__main__":
    breed_info = get_wiki_breed_summary("Affenpinscher")
    if breed_info:
        print("Title:", breed_info['title'])
        print("Description:", breed_info['description'])
        print("Extract:", breed_info['extract'])
        print("Thumbnail:", breed_info['thumbnail'])
        print("Original Image:", breed_info['original_image'])

Title: Affenpinscher
Description: German breed of dog
Extract: The Affenpinscher is a German breed of small toy dog of Pinscher type. Originally developed as a mouser, the Affenpinscher is now commonly found as a companion worldwide.
Thumbnail: https://upload.wikimedia.org/wikipedia/commons/thumb/1/17/Affenpinscher.jpg/320px-Affenpinscher.jpg
Original Image: https://upload.wikimedia.org/wikipedia/commons/1/17/Affenpinscher.jpg


In [10]:
def display_pet_friendly_places(title, df):
    print(f"\n{title}:")
    if df.empty:
        print("No data available.")
    else:
        print(df.to_string(index=False))

In [ ]:
def display_dog_breed_info(breed):
    breed_lower = breed.lower()
    if breed_lower in DOG_BREED_ATTRIBUTES:
        info = DOG_BREED_ATTRIBUTES[breed_lower]
        print(f"\n🐶 Breed Insights for {breed}:")
        print(f"   Energy Level: {info['energy']}")
        print(f"   Size: {info['size']}")
        print(f"   Notes: {info['notes']}")
        print("-" * 50)
    else:
        print(f"\nNo specific breed info available for '{breed}'.")
        print("-" * 50)

In [12]:
def pawfect_planner(user_location, breed=None):
    print(f"\n📍 Planning for **{user_location}**" + (f" with {breed}" if breed else "") + "...")

    # ✅ Weather
    weather_info = get_weather(user_location)
    print(f"\n🌦️ **Weather in {weather_info['City']}**")
    print(f"   🌡️ Temperature: {weather_info['Temperature']}°C")
    print(f"   ☁️ Condition: {weather_info['Weather']}")
    print(f"   💨 Wind Speed: {weather_info['Wind Speed']} km/h")
    print(f"   💡 Recommendation: {weather_info['Recommendation']}")
    
    if breed:
        breed_info = get_wiki_breed_summary(breed)
        if breed_info:
            print(f"\n🐶 **Breed Info for {breed}:**")
            print(f"Title: {breed_info['title']}")
            print(f"Description: {breed_info['description']}")
            print(f"Extract: {breed_info['extract']}")
            if breed_info['thumbnail']:
                print(f"Thumbnail URL: {breed_info['thumbnail']}")
            print("-" * 50)
        else:
            print(f"No summary found for '{breed}'.")
            print("-" * 50)


    # ✅ Places
    places_df = get_pet_friendly_places(user_location)
    display_pet_friendly_places("Top Pet-Friendly Places", places_df)

    # ✅ Events
    events_df = get_pet_events(user_location)
    display_pet_friendly_places("Top Pet Events", events_df)

In [13]:
# 🚀 **Run the Program**
if __name__ == "__main__":
    user_location = input("\n🔎 Enter city or ZIP code: ")
    user_breed = input("\n🐶 Enter dog breed (or press Enter to skip): ").strip() or None
    pawfect_planner(user_location, user_breed)


📍 Planning for **94103** with Golden Retriever...

🌦️ **Weather in San Francisco**
   🌡️ Temperature: 17.62°C
   ☁️ Condition: Broken clouds
   💨 Wind Speed: 3.09 km/h
   💡 Recommendation: 🌥️ Cloudy & Mild: Great for both indoor and outdoor activities.

🐶 **Breed Info for Golden Retriever:**
Title: Golden Retriever
Description: Scottish breed of dog
Extract: The Golden Retriever is a Scottish breed of retriever dog of medium size. It is characterised by a gentle and affectionate nature and a striking golden coat. It is a working dog, and registration is subject to successful completion of a working trial. It is commonly kept as a pet and is among the most frequently registered breeds in several Western countries; some may compete in dog shows or obedience trials, or work as a guide dog.
Thumbnail URL: https://upload.wikimedia.org/wikipedia/commons/thumb/b/bd/Golden_Retriever_Dukedestiny01_drvd.jpg/320px-Golden_Retriever_Dukedestiny01_drvd.jpg
------------------------------------------